In [ ]:
%%time
import time
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy as mathlib
import scipy.optimize
import math

 # choose from [1,3,4,6]
landcover_index=6

if landcover_index==1:
    landcover_beginfile=[12]# shrublands
    initial_guess_raw_smap=[0.05,0.11,0.11,1.5]
    initial_guess_raw_ulsca=[0.05,0.14,0.08,1]
    print("shrublands")
elif landcover_index==2:
    print("tree")
elif landcover_index==3:
    landcover_beginfile=[4,7,8,13,14]# grass
    initial_guess_raw_smap=[0.05,0.13,0.156,1.5]
    initial_guess_raw_ulsca=[0.04,0.12,0.18,1.5]
    print("grass")
elif landcover_index==4:
    landcover_beginfile=[2,3,5,6,11,16,19,22,23,24,25,26,27,28,30,32,34,36,38,39,40,43,44]# crop
    initial_guess_raw_smap=[0.05,0.11,0.094,3.5]
    initial_guess_raw_ulsca=[0.07,0.1,0.08,0.5]
    print("crop")
elif landcover_index==5:
    print("cropandnatural")
elif landcover_index==6:
    landcover_beginfile=[9,10]# bareland
    initial_guess_raw_smap=[0,0,0.15,0]
    initial_guess_raw_ulsca=[0,0,0.05,0]
    print("bareland")

# choose one ：smap or ulsca
omega=initial_guess_raw_ulsca[0]
b=initial_guess_rawulsca[1]
h=initial_guess_rawulsca[2]
StemFactor=initial_guess_rawulsca[3]
tb_bias=30

plandcover_beginfile = [i-1 for i in landcover_beginfile]
for myindex in plandcover_beginfile:
    nameall = np.genfromtxt("./matchfile_NEW.csv",delimiter=",",skip_header=1,dtype=str,usecols=range(2),invalid_raise=False,filling_values=np.nan)
    name=nameall[myindex,0].astype(str)   # get drone latitude
    fpath = "./draw UAV/"+name+"/"
    radfile =  name+".dat"
    flogfile = nameall[myindex,1].astype(str)   # get drone longitude
    # flogfile = name+"_p31gps.csv"
    print(name)
    #### v2.csv is Litchi's flightlog ####
    if flogfile.endswith('v2.csv') or flogfile.endswith('V2.CSV'):
        logtype="Litchi"
        UTCerror = 0
        #call function to load and convert Litchi format flightlog
        #print("Detected .csv format flightlog, proceeding with Litchi format.")

        flog = np.genfromtxt(fpath+flogfile,delimiter=",",skip_header=1,dtype=str,usecols=range(67),invalid_raise=False,filling_values=np.nan)
        datestrings = flog[:,11] # OR COLUMN 12 is Local
        tms = flog[:,10].astype(float)  # drone flight ticker time in milliseconds
        lat = flog[:,0].astype(float)   # get drone latitude
        lon = flog[:,1].astype(float)   # get drone longitude
        alt = flog[:,2].astype(float)   #altitude above ground (m)
        speed = flog[:,4].astype(float)
    #     flystate = flog[:,66].astype(float)
        pitch = flog[:,22].astype(float)
        roll = flog[:,23].astype(float)
        yaw = flog[:,24].astype(float)
        dronet = pd.to_datetime(datestrings)+datetime.timedelta(hours=0)

        #print(dronet[-1])
        dronet=dronet+datetime.timedelta(seconds=0)+datetime.timedelta(hours=0)
        dronetnum = (dronet - datetime.datetime(1970, 1, 1)) / datetime.timedelta(seconds=1.)
        lonraw = lon
        latraw = lat
        #print("")
        #print("Drone Start Time:")
        #print(dronet[0])
        #print("")
        #print("Drone End Time:")
        #print(dronet[-1])
        #print("")
        #print("Done.")

    #### p31gps.csv is the ending for vehcile-based systems. ####
    if flogfile.endswith('p31gps.csv') or flogfile.endswith('P31GPS.CSV') or flogfile.endswith('P31GPS.csv'):
        logtype="P31GPS"
        UTCerror = 0
        #call function to load and convert Litchi format flightlog
        print("Detected Ground-based GPS Telemetry file.")

        flog = np.genfromtxt(fpath+flogfile,delimiter=",",skip_header=1,dtype=str,invalid_raise=False,filling_values=np.nan)
        day = flog[:,0].astype(int)    # drone flight ticker time in milliseconds
        month = flog[:,1].astype(int)  # get drone latitude
        year = flog[:,2].astype(int)   # get drone longitude
        hour = flog[:,3].astype(int)   #altitude above ground (m)
        minute = flog[:,4].astype(int)
        second = flog[:,5].astype(int)
        lat = flog[:,6].astype(float)
        lon = flog[:,7].astype(float)
        altabs = flog[:,8].astype(float)
        speed = flog[:,9].astype(float)

        course = flog[:,10].astype(float)
        HDOP = flog[:,11].astype(float)
        numsat = flog[:,12].astype(int)

        alt = altabs-np.mean(altabs[0:10])

        pitch = np.zeros_like(lat)
        roll = np.zeros_like(lat)
        yaw = np.zeros_like(lat)

        #dronet=dronet+datetime.timedelta(seconds=0.0)+datetime.timedelta(hours=0)
        #dronetnumdelta = datetime.datetime(year,month,day,hour,minute,second)-datetime.datetime(1970, 1, 1,0,0,0)
        datearr      = np.column_stack((year,month,day,hour,minute,second))
        dt_ref    = datetime.datetime(1970, 1, 1,0,0,0)
        dronetnum  = np.array([(datetime.datetime(y,m,d,h,mi,s)-dt_ref).total_seconds() for y,m,d,h,mi,s in datearr])

        dronet  = [datetime.datetime.utcfromtimestamp(ts) for ts in dronetnum]
        #dronetnum = dronetnumdelta.total_seconds()
        lonraw = lon
        latraw = lat

        print("")
        print("GPS file Start Time:")
        print(dronet[0])
        print("")
        print("GPS file End Time:")
        print("")
        print(dronet[-1])

        print("Done.")


    #print("Processing Flightlog Data...",end='',flush=True) 
    #print("Done.")

    #print("maxlat:",np.nanmax(lat))
    #print("minlat:",np.nanmin(lat))

    #print("maxlon:",np.nanmax(lon))
    #print("minlon:",np.nanmin(lon))

    if logtype=="P31GPS":
        alt = 1.5*np.ones_like(alt) # m above ground for fixed (vehicle-based) sensor
    #print("Loading PoLRa Data...",end='',flush=True)
    DATA = np.genfromtxt(fpath+radfile,delimiter=None,autostrip=True)
    #print("Done.")
    if 'polra2bool' in locals():  # POLRA2 had different channels
        if polra2bool == 'True':
            ml = DATA[:,5]     #matched load
            cl = DATA[:,6]     #cold load
            rv = DATA[:,7]     #v pol
            rh = DATA[:,8]     #h pol
            Tdet = DATA[:,9]   #detector physical temp
            Tml = DATA[:,10]    #matched load physical temp
            Tcl = DATA[:,11]    #cold load physical temp
            Tpam1 = DATA[:,12]  #external/antenna temp 1
            Tpam2 = DATA[:,13]  #external / antenna temp 2
            Tpam = (Tpam1+Tpam2)/2
            uml = DATA[:,14]   #matched load
            ucl = DATA[:,15]   #cold load
            urv = DATA[:,16]   #v pol
            urh = DATA[:,17]   #h pol

    else:  # standard column format since PoLRa3
        ###radiometer voltages
        cl = DATA[:,5]     #cold load
        ml = DATA[:,6]     #matched load
        rv = DATA[:,7]     #v pol
        rh = DATA[:,8]     #h pol

        ###extract physical temperatures and save mean to vector
        Tdet = DATA[:,9]   #detector physical temp (null for PoLRa3)
        Tml = DATA[:,10]    #matched load physical temp
        Tcl = DATA[:,11]    #cold load physical temp
        Tpam1 = DATA[:,12]  #aux board temp / ext. 1
        Tpam2 = DATA[:,13]  #external temp
        Tpam = (Tpam1+Tpam2)/2

        ucl = DATA[:,14]   #cold load
        uml = DATA[:,15]   #matched load
        urv = DATA[:,16]   #v pol
        urh = DATA[:,17]   #h pol
    # apply a time offset if defined in config file
    if 'toffset' not in locals():
        #print("no time offset")
        toffset = 0

    unixtime = DATA[:,4].astype(float)-UTCerror*3600+toffset
    radtnum = np.array(unixtime)
    drntnum = np.array(dronetnum)
    latrad = np.interp(radtnum,drntnum,lat,left=np.nan,right=np.nan)
    lonrad = np.interp(radtnum,drntnum,lon,left=np.nan,right=np.nan)
    altrad = np.interp(radtnum,drntnum,alt,left=np.nan,right=np.nan)# flight height
    #altabsrad = np.interp(radtnum,drntnum,altabs,left=np.nan,right=np.nan)

    def getTCL(hclfile,vclfile):
        hpars = np.genfromtxt(hclfile)
        vpars = np.genfromtxt(vclfile)
        return hpars,vpars
    hfile = './TclH.txt'         # files containing fit parameters for cold load TB
    vfile = './TclV.txt'
    hpars,vpars = getTCL(hfile,vfile)

    #print("Retrieving PoLRa Data...",end='',flush=True)
    Tcl_H = hpars[0]*Tcl+hpars[1] #0.335*Tcl-90
    Tcl_V = vpars[0]*Tcl+vpars[1] #0.335*Tcl-90

    slopeh=(Tml-Tcl_H)/(ml-cl)
    offh = slopeh*(-ml)+Tml
    Tb_h = offh+rh*slopeh                   #calibrated h pol

    slopev=(Tml-Tcl_V)/(ml-cl)
    offv = slopev*(-ml)+Tml
    Tb_v = offv+rv*slopev

    if landcover_index==3:
        #cali tb
        Tb_h = Tb_h + tb_bias
        Tb_v = Tb_v + tb_bias
        #cali tb


    cleanTB_h=Tb_h
    cleanTB_v=Tb_v
    stdthresh=1
    cleanTB_h[urv>stdthresh]=np.nan  #apply standard deviation filter
    cleanTB_h[urh>stdthresh]=np.nan  #both polarizations filtered together
    cleanTB_v[urv>stdthresh]=np.nan  #apply standard deviation filter
    cleanTB_v[urh>stdthresh]=np.nan  #both polarizations filtered together
    cleanTB_v[np.isnan(cleanTB_h)]=np.nan   #make sure we have both Tbs or both NaN
    cleanTB_h[np.isnan(cleanTB_v)]=np.nan   #make sure we have both Tbs or both NaN

    # lat[np.isnan(cleanTB_v)]=np.nan
    # lon[np.isnan(cleanTB_v)]=np.nan
    # print(cleanTB_h,cleanTB_v)
#     plt.figure()
#     plt.plot(cleanTB_h)
#     plt.plot(cleanTB_v)
#     plt.legend(['TB_h','TB_v'])

    tsec = unixtime-unixtime[0]
    tmin = tsec/60

#     plt.figure()
#     plt.plot(tmin,ml)
#     plt.plot(tmin,cl)
#     # plt.plot(tmin,rv)
#     # plt.plot(tmin,rh)
#     plt.ylabel('Detector Voltage (mV)')
#     plt.legend(['Matched Load','Cold Load'])
#     # plt.legend(['Matched Load','Cold Load','Antenna V','Antenna H'])
#     plt.title('Raw Voltages')
#     plt.grid()

#     plt.figure()
#     plt.plot(tmin,Tml,'-o')
#     plt.plot(tmin,Tcl,'-o')
#     plt.plot(tmin,Tpam,'-o')
#     plt.ylabel('Physical Temperatures (K)')
#     plt.legend(['Matched Load','Cold Load','Antenna','Detector'])
#     plt.grid()
    #print("Done.")


    NDVIref=0.851 # annual max NDVI
    NDVI=0.4 
    T_g=23+273.15 # surface temperature

    ps=1.51 # bulk density
    sand=4 # 100%
    clay=20 # 100%
    # flogfile

    supp_fpath = "./supplement_data.csv" #old temperature 'TB_soil_oldt.csv'
    supp = pd.read_csv(supp_fpath)
    # print(supp)
    supp_name=supp["filename"].astype(str) 
    supp_namew=supp["wfilename"].astype(str) 
    NDVIref_all = supp["NDVI_ref"].astype(float) 
    NDVI_all = supp["NDVI"].astype(float) 
    T_s_all = supp["Soil_temperature_level1"].astype(float)
    T_v_all = supp["skin_temperature(vegetation_t)"].astype(float)
    T_w_all = supp["ground_temperature"].astype(float)
    ps_all = supp["bulk_densi"].astype(float)
    sand_all = supp["sand100%"].astype(float) 
    clay_all = supp["clay100%"].astype(float)   
    mode_all=supp["landuse"].astype(str)
    w_Tb_all = supp["w_Tb"].astype(float)
    timeflag_all= supp["timeflag"].astype(str)
    slope_all = supp["slope20"].astype(float)  
    index=np.where(supp_name==flogfile)
    #print(index[0].astype(int))

    if np.array(timeflag_all[index[0]])[0]=="winter":
        mode="bareland"
    else:
        mode=np.array(mode_all[index[0]])[0]

    # if mode=="crop" or mode=="grass" or mode=="cropandnatural":
    #     NDVIref=np.array(NDVI_all[index[0]])[0]
    # else:
    #     NDVIref = np.array(NDVIref_all[index[0]])[0]


    # NDVIref = np.array(NDVIref_all[index[0]])[0]
    NDVI = np.array(NDVI_all[index[0]])[0]
    T_s = np.array(T_s_all[index[0]])[0]
    T_v = np.array(T_v_all[index[0]])[0]
    T_w = np.array(T_w_all[index[0]])[0]
    w_Tb = np.array(w_Tb_all[index[0]])[0]
    ps = np.array(ps_all[index[0]])[0]
    sand = np.array(sand_all[index[0]])[0]
    clay = np.array(clay_all[index[0]])[0]
    slope = np.array(slope_all[index[0]])[0]

    slopefile = pd.read_csv(fpath+"Export_Output.txt", sep=',') 
    
    viewangle=40.0
    radtnum = np.array(unixtime)
    drntnum = np.array(dronetnum)
    rollradt = np.interp(radtnum,drntnum,roll,left=np.nan,right=np.nan)
   
    #careful with mount orientation
    alpha = viewangle-rollradt-theta_bias          #incidence angle after roll correction
    # nominal mount direction
    #print("mean look angle:",np.nanmean(alpha))

    ##### Apply cable loss correction
    loss_ratio = 0.5 #may depend on system! (whitey had therm outside Alu)
    Teff = loss_ratio*Tpam+(1-loss_ratio)*Tml #effective temperature of transmission line loss
    #print("Applying Cable Loss Correction...",end='',flush=True)
    Loss_V = vpars[2]
    Loss_H = hpars[2]
    t_cablev = 10**(Loss_V/10)
    t_cableh = 10**(Loss_H/10)
    cleanTB_h = (cleanTB_h-(1-t_cableh)*Teff)/(t_cableh)
    cleanTB_v = (cleanTB_v-(1-t_cablev)*Teff)/(t_cablev)
    raw_cleanTB_h = cleanTB_h
    raw_cleanTB_v = cleanTB_v
    #print("done")

    ## ew
    ew1=80-0.4*(T_w-293.15) 

    def PureWater(T, freq):

        """
        The dielectric constant and the complex refractive index of pure water.
        Background information: Meissner, Thomas, and Frank J. Wentz."The complex dielectric constant of pure and sea water from microwave satellite observations."
        IEEE Transactions on Geoscience and remote Sensing 42.9 (2004): 1836-1849.
        Epsilon_water, Kw2 = PureWater(T, freq)
        T:          temperature [Centigrade]
        freq:       frequency [GHz]
        """
        Epsilon_s = (3.70886 * pow(10, 4) - 8.2168 * pow(10, 1) * T) \
                    / (4.21854 * pow(10, 2) + T)
        Epsilon_1 = 5.7230 + (2.2379 * pow(10, -2) * T) + (-7.1237 * pow(10, -4) * T ** 2)
        Epsilon_inf = 3.6143 + (2.8841 * pow(10, -2) * T)
        Nu_1 = (45 + T) / (5.0478 + (-7.0315 * pow(10, -2) * T) + (6.0059 * pow(10, -4) * T ** 2))
        Nu_2 = (45 + T) / (1.3652 * pow(10, -1) + (1.4825 * pow(10, -3) * T) + (2.4166 * pow(10, -4) * T ** 2))

        Epsilon_water = (Epsilon_s - Epsilon_1) / (1 + 1j * freq / Nu_1) + \
                        (Epsilon_1 - Epsilon_inf) / (1 + 1j * freq / Nu_2) + Epsilon_inf

        Kw2 = np.abs((Epsilon_water - 1) / (Epsilon_water + 2)) ** 2
        return Epsilon_water, Kw2

    ew2=PureWater(T_w-273.15, 1.47)
    ew3=ew2[0]
    ew=ew3.real
    #print(ew3.real)
    # Dobson 
    import numpy as np
    sm=np.arange(0,1.01,0.01)


    def reverse_DielDobson (wc, ew, sand, clay):
        # REAL : wc, sand, clay
        # COMPLEX : ew, eps
        # REAL : rho_s, rho_b, eps_s, beta, eaa,  epsr, epsi
        # REAL, PARAMETER : alphas = 0.65

        alphas = 0.65
        rho_s = 2.66
        rho_b = (sand*1.6 +  clay*1.1 + (100.-sand-clay)*1.2)/100.
        eps_s = (1.01 + 0.44 * rho_s)**2. - 0.062  # compare to ATBD: 4.7
        wc =np.max(wc)*1.000 # to avoid dividing by zero

        beta = (127.48 - 0.519 * sand - 0.152 * clay) / 100.
        eaa = 1.0 + (rho_b / rho_s) * (eps_s ** alphas - 1.0)+ (wc ** beta) * (float(ew) ** alphas) - wc
        epsr = eaa ** (1./alphas) #实部
        return epsr



    epsr = [[0] * len(sm) for _ in range(len(cleanTB_h))]
    for i in range(len(cleanTB_h)):
        for j in range(len(sm)):
            if np.isnan(cleanTB_h[i]) | np.isnan(alpha[i]):
                epsr[i][j] = np.nan
            else:
                epsr[i][j]=reverse_DielDobson(sm[j], ew, sand, clay)
    #             print("i:",i,"j:",j)
    #print("done")
   
    epsr = np.array(epsr)

    ## Define trig functions in degrees
    def cosd(deg):
        rad=np.cos(deg*np.pi/180)
        rad=np.array(rad)
        return rad
    def sind(deg):
        rad=np.sin(deg*np.pi/180)
        rad=np.array(rad)
        return rad
    def tand(deg):
        rad=np.tan(deg*np.pi/180)
        rad=np.array(rad)
        return rad
    # tau-omega
    VWC = (1.9134*(NDVI**2) - 0.3215*NDVI) + StemFactor*(NDVIref - 0.1) / (1 - 0.1)
    tau= b*VWC


    e_surf_v=[]
    e_soil_v=[]
    e_surf_h=[]
    e_soil_h=[]
    cleanTB_h_retrl=[]
    cleanTB_v_retrl=[]

    for i in range(101):
        ## fresnel
        e_soil_h.append((1-(abs((cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                                (cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2))
        e_soil_v.append((1-(abs((epsr[:,i]*cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                             (epsr[:,i]*cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2))
        ## roughness
        e_surf_h.append((1-(abs((cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                                (cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha))))
        e_surf_v.append((1-(abs((epsr[:,i]*cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                             (epsr[:,i]*cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha))))


        ## tau-omega
        cleanTB_h_retrl.append((1-(abs((epsr[:,i]*cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                             (epsr[:,i]*cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha)))*T_s*np.exp(-tau/cosd(alpha))+
                               (1-omega)*T_v*(1-np.exp(-tau/cosd(alpha)))*
                               (1+(1-(1-(abs((epsr[:,i]*cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                             (epsr[:,i]*cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha))))*np.exp(-tau/cosd(alpha))))    
        cleanTB_v_retrl.append((1-(abs((cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                                (cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha)))*T_s*np.exp(-tau/cosd(alpha))+
                               (1-omega)*T_v*(1-np.exp(-tau/cosd(alpha)))*
                               (1+(1-(1-(abs((cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                                (cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha))))*np.exp(-tau/cosd(alpha))))

    #print('done')
    df_h=pd.DataFrame()
    df_v=pd.DataFrame()
    for i in range(101):   
        ## tau-omega
        frames_h = pd.DataFrame(pd.Series(((1-(abs((epsr[:,i]*cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                             (epsr[:,i]*cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha)))*T_s*np.exp(-tau/cosd(alpha))+
                               (1-omega)*T_v*(1-np.exp(-tau/cosd(alpha)))*
                               (1+(1-(1-(abs((epsr[:,i]*cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                             (epsr[:,i]*cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha))))*np.exp(-tau/cosd(alpha))))),columns=[str(i)])
        df_h=pd.concat([df_h,frames_h], axis=1)
        frames_v = pd.DataFrame(pd.Series(((1-(abs((cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                                (cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha)))*T_s*np.exp(-tau/cosd(alpha))+
                               (1-omega)*T_v*(1-np.exp(-tau/cosd(alpha)))*
                               (1+(1-(1-(abs((cosd(alpha)-np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))/
                                (cosd(alpha)+np.sqrt(epsr[:,i]-sind(alpha)*sind(alpha)))))**2)/(np.exp(-h*cosd(alpha)*cosd(alpha))))*np.exp(-tau/cosd(alpha))))),columns=[str(i)])
        df_v=pd.concat([df_v,frames_v], axis=1)       
    #print("done")

    position_a=[]
    for i in range(len(cleanTB_h)):  
        if np.isnan(cleanTB_h[i]):
            position_a.append(np.nan)
        else:
            min_value = np.nanmin(abs(cleanTB_h[i]-df_h.iloc[i]))
            if np.isnan(min_value):
                position_a.append(np.nan)
            else:
                min_index = np.where(abs(cleanTB_h[i]-df_h.iloc[i]) == min_value)
    #             print(min_index[0])
                aaa=min_index[0]*0.01
                position_a.append(aaa[0])

    ##change GPS
    def millerToXY (lon,lat):
        xy_coordinate =[]      
        L= 6381372*math.pi*2
        W=L
        H=L/2
        mill =2.3
        x=lon*math.pi/180
        y = lat*math.pi/180
        y= 1.25*math.log(math.tan(0.25*math.pi+0.4*y))
        x = (W/2)+(W/(2*math.pi))*x
        y= (H/2)-(H/(2*mill))*y
        xy_coordinate.append((int(round(x)),int(round(y)))) 
        return xy_coordinate

    def xy_to_coor(x,y):
        lonlat_coordinate =[] 
        L =6381372 * math.pi*2 
        W=L 
        H=L/2 
        mill =2.3
        lat = ((H/2-y)*2*mill)/(1.25*H)
        lat=((math.atan(math.exp(lat))-0.25*math.pi)*180)/(0.4*math.pi) 
        lon =(x-W/2)*360/W
        lonlat_coordinate.append((round(lon,15),round(lat,15))) 
        return lonlat_coordinate

    latrad_p=latrad;
    lonrad_p=lonrad;
    for i in range(len(cleanTB_h)-1): 
        if np.isnan(lonrad[i]) | np.isnan(latrad[i]) | np.isnan(lonrad[i+1]) | np.isnan(latrad[i+1]):
            distance_h=0;
        else:
            distance_h=altrad[i]*0.8391/1000;
            resultxy1=millerToXY (lonrad[i],latrad[i])
            xrad1=resultxy1[0][0]
            yrad1=resultxy1[0][1]
            resultxy2=millerToXY (lonrad[i+1],latrad[i+1])
            xrad2=resultxy2[0][0]
            yrad2=resultxy2[0][1]
            if (xrad2-xrad1)==0: 
                xrad0=xrad1
                if yrad2>yrad1:
                    yrad0=yrad1+distance_h
                else:
                    yrad0=yrad1-distance_h
                xy0=xy_to_coor(xrad0,yrad0)
                lonrad0=xy0[0][0]
                latrad0=xy0[0][1]
                lonrad_p[i]=lonrad0
                latrad_p[i]=latrad0
            else:
                line_k=(yrad2-yrad1)/(xrad2-xrad1)
                line_b=yrad2-line_k*xrad2
                if xrad2>xrad1:
                    xrad0=math.sqrt(distance_h*distance_h/(1+line_k*line_k))+xrad1
                else:
                    xrad0=-math.sqrt(distance_h*distance_h/(1+line_k*line_k))+xrad1
                yrad0=line_k*xrad0+line_b    
                xy0=xy_to_coor(xrad0,yrad0)
                lonrad0=xy0[0][0]
                latrad0=xy0[0][1]
                lonrad_p[i]=lonrad0
                latrad_p[i]=latrad0
#     print("done")

    df=pd.DataFrame({
        "TB_V":cleanTB_v,
        "TB_H":cleanTB_h,
        "raw_TB_V":raw_cleanTB_v,
        "raw_TB_H":raw_cleanTB_h,
        "lat":latrad,
        "lon":lonrad,
        "latp":latrad_p,
        "lonp":lonrad_p,
        "soil_moisture":position_a,
        "alt":altrad
    })

    dfpath=fpath+r'TB_soil_oldt.csv'
    df.to_csv(dfpath,encoding='utf-8')